In [ ]:
using Primes
using Printf
using ProgressBars
using Mods

# Тест Ферма

https://rosettacode.org/wiki/Fermat_numbers#Julia


In [ ]:
"""
Вычисление символа Якоби \$\\left(\\frac{a}{n}\\right)\$

    jakobi(a, n)

[Реализация](https://rosettacode.org/wiki/Jacobi_symbol#Julia)
"""
function jacobi(a, n)
    a %= n
    result = 1
    while a != 0
        while iseven(a)
            a ÷= 2
            ((n % 8) in [3, 5]) && (result *= -1)
        end
        a, n = n, a
        (a % 4 == n % 4 == 3) && (result *= -1)
        a %= n
    end
    return n == 1 ? result : 0
end
;

In [ ]:
"""
Тест Ферма с длинной арифметикой
"""
function fpprime(n; iterations = 10, only_consider_coprimes=false)
    for a in rand(2:n-1, iterations)
        if only_consider_coprimes && gcd(n, a) != 1
            # Вообще, если gcd(n, a) != 1, можно и сразу false вернуть, но gcd(n, a) здесь не для этого,
            # а как раз для того, чтобы помочь тесту Ферма обмануться (см. ниже числа Кармайкла).
            # В "боевом" режиме таким пользоваться конечно нельзя, т.к. gcd небыстрый.
            continue
        end
        if mod(big(a)^(n - 1), n) != 1
            return false
        end
    end
    return true
end

"""
Тест Ферма без явной длинной арифмметики, но потенциально бесполезный =)
"""
function fpprimem(n; iterations = 10, only_consider_coprimes=false)
    for a in rand(2:n-1, iterations)
        if only_consider_coprimes && gcd(n, a) != 1
            # Аналогично, если gcd(n, a) != 1, можно и сразу false вернуть, но gcd(n, a) здесь не для этого,
            # а как раз для того, чтобы помочь тесту Ферма обмануться (см. ниже числа Кармайкла).
            # В "боевом" режиме таким пользоваться конечно нельзя, т.к. gcd небыстрый.
            continue
        end
        if powermod(a, n - 1, n) != 1 # Mod(a, n) ^ (n - 1) != 1  # Slooooow
            return false
        end
    end
    return true
end
;

In [ ]:
for n in ProgressBar(3:3000000)
    if fpprimem(n) != Primes.isprime(n)
        println("Fail on $(n)")
    end
end

## Числа Кармайкла

**Число Кармайкла** — составное число $n$, которое удовлетворяет $а^{n-1}\equiv 1\pmod{n}$ для всех целых $а$, взаимно простых $n$

Первые — 561, 1105, 1729, 2465, 2821, 6601, 8911, 10585, 15841, 29341, 41041, 46657, 52633, 62745, 63973, 75361. Если среди случайных не встретилось не взаимно простых с числами Кармайкла, тест Ферма на них ошибается

In [ ]:
first_сarmichael_numbers = [561, 1105, 1729, 2465, 2821, 6601, 8911, 10585, 15841, 29341, 41041, 46657, 52633, 62745, 63973, 75361]

for n in first_сarmichael_numbers
    println("На $(n) $(if fpprimem(n, only_consider_coprimes=false) "" else "не " end) ошибся.")
end

In [ ]:
"""
Тест Соловея-Штрассена

    sstprimem(n; [iterations])

сообщает, что `n` — простое с вероятностью \$1 - 2^{-\\mathit{iterations}}\$
"""
function sstprimem(n; iterations = 10)
    if iseven(n)
        return false
    end

    for a in rand(2:n-1, iterations)
        x = jacobi(a, n)
        if x == 0 || powermod(a, (n-1)÷2, n) != mod(x, n) #  mod(big(a) ^ ((n - 1)÷2) - x, n) != 0
            return false
        end 
    end
    return true
end
;

In [ ]:
for n in ProgressBar(3:76000)
    if sstprimem(n) != Primes.isprime(n)
        println("Fail on $(n)")
    end
end

In [ ]:
for _ in 1:10000
    for n in first_сarmichael_numbers
        if sstprimem(n)
            println("Тест Соловея-Штрассена ошибся на $(n)!!!")
        end
    end
end